# Mini Project 3: COVID-19 Data Analysis and Machine Learning

#### Created by Group 7 - Kamilla, Jeanette, Juvena

## Objective

This assignment aims to develop practical skills in data analysis, visualization, and machine learning using real-world COVID-19 data. The project focuses on exploring global pandemic-related indicators to uncover trends, build predictive models, and apply both supervised and unsupervised learning techniques using Python.

Before we begin analyzing the COVID-19 dataset, we need to import a few essential Python libraries that will help us manipulate the data, build models, and visualize our findings:

- **Pandas**: This is a powerful library used to handle and manipulate data in tables (called DataFrames).
- **NumPy**: It helps with numerical operations, especially when we work with arrays or need to do math.
- **Matplotlib** and **Seaborn**: These are popular libraries for creating visual charts and graphs. We'll use them to help us understand the data better by seeing it.
- **SciPy (stats module)**: This gives us access to statistical tools like checking if data is normally distributed.

We'll also configure default styles for our plots to ensure they're clean, visually appealing, and easy to interpret.

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import metrics
import sklearn.metrics as sm
from scipy.spatial.distance import cdist
from sklearn import preprocessing as prep
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score





# Set plot styles for better visualization
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

--------

# 1. Data wrangling and exploration

### 1.1 Load the Data

Now that we have our tools ready, the next step is to load the COVID-19 dataset into Python so we can start analyzing it.

In this case, we’re working with a single dataset:

- **OWID COVID-19 Latest Data**: a CSV file that contains country-level information on cases, deaths, vaccinations, testing, and various socioeconomic indicators.

We'll use Pandas to read the CSV file and store it as a DataFrame. To make our code cleaner and reusable, we'll define a simple function that loads the data and performs some initial checks. This way, we can easily reload or replace the dataset if needed in future steps.

In [ ]:
# File paths for the covid datasets. (dataset: last updated 2024-08-04)
dataset_covid = 'Dataset/owid-covid-latest.csv'

# Function to load the Excel files
def load_csv_to_dataframe(file_path):
    # Reads the Excel file and skips the first row if it contains a description or title
    df = pd.read_csv(file_path)
    return df

# Load datasets
print("..Loading COVID-19 dataset")
df_covid = load_csv_to_dataframe(dataset_covid)

### 1.2 Explore the Data

After loading the dataset, we want to explore it to understand what kind of information it contains and how it's structured.

To do this, we can use several helpful Pandas functions such as `shape`, `types`, `info()`, `head()`, `tail()`, `sample()`, `describe()` and `isnull().sum()`. These functions will give us insights into the number of rows and columns, the data types of each column, a summary of the data, and any missing values. 

This exploration is crucial as it helps us identify potential issues or areas that need further cleaning or transformation before we proceed with our analysis. 

In [ ]:
# Check the shape of the DataFrame (rows, columns)
df_covid.shape

In [ ]:
# Display the types of attributes (colum names) in the DataFrame
df_covid.dtypes

In [ ]:
# Gives an overview of the DataFrame
df_covid.info()

In [ ]:
# Display the first 5 rows of the DataFrame
df_covid.head()

In [ ]:
# Display the last 5 rows of the DataFrame
df_covid.tail()

In [ ]:
# Display a random sample of 5 rows from the DataFrame
df_covid.sample(5)

In [ ]:
# Gives summary statistics for all numerical columns in the dataset
df_covid.describe()

##### **1.2.1 Summary of exploring the data**

After exploring the dataframe, we found that it contains a large number of columns, many of which are not useful for our analysis or modeling goals. While some columns provide valuable information (like total cases, deaths, and vaccination rates), others are either redundant, mostly empty, or irrelevant.

This highlights the need for a thorough data cleaning step to remove unnecessary columns, handle missing values, and focus only on the most relevant features for our machine learning tasks.

### 1.3 Clean the Data

After loading and exploring the data, we need to clean it to ensure that our analysis is accurate and meaningful. Data cleaning involves several steps, including: checking for missing values, removing duplicates, and converting data types.

In [ ]:
# Check for missing values in the DataFrame
df_covid.isnull().sum()

The output above shows that many columns contain no values at all, so we will remove them to clean up the dataset.

In [ ]:
# Before cleaning the data, we want to remove irrelevant OWID aggregate rows—such as those representing high-income, low-income, and other income groupings.
rows_to_remove = ["OWID_UMC", "OWID_WRL", "OWID_LMC", "OWID_LIC", "OWID_HIC"]
df_removed_rows = df_covid[~df_covid["iso_code"].isin(rows_to_remove)]

We are removing the 'low-income countries', 'lower-middle-income countries', 'upper-middle-income countries', 'high-income countries' and 'world' categories because they are too broad and lack specific country-level detail, making it difficult to draw meaningful conclusions without relying on assumptions.

In [ ]:
# Checking if the above rows were removed
print(f"{df_covid.shape}")
print(f"Removed the {df_covid.shape[0] - df_removed_rows.shape[0]} OWID rows from the dataframe.")

In [ ]:
# We will drop all columns with no values at all like; excess_mortality_cumulative_absolute, excess_mortality_cumulative etc.
df_covid_removed_columns = df_removed_rows.dropna(axis=1, how='all')

In [ ]:
# Check whether the columns were removed
print(f"COVID dataframe shape after removing columns: {df_covid_removed_columns.shape}")
print(f"Removed {df_covid.shape[1] - df_covid_removed_columns.shape[1]} columns from the dataframe.")

We have selected a subset of columns that we consider relevant for our analysis. These columns include key features related to COVID-19 cases, deaths, vaccinations, demographics, and health indicators. By keeping only these columns, we focus on the most informative data for building meaningful models and drawing accurate insights, while reducing noise and unnecessary complexity in the dataset.

In [ ]:
# We make a new dataframe with the columns we want to keep for future analysis.
columns_we_want_to_keep = [
    "iso_code", "continent", "location", "total_cases", "total_deaths",
    "total_cases_per_million", "total_deaths_per_million",
    "total_vaccinations", "people_vaccinated", "people_fully_vaccinated",
    "total_boosters", "new_vaccinations", "new_vaccinations_smoothed",
    "total_vaccinations_per_hundred", "people_vaccinated_per_hundred",
    "people_fully_vaccinated_per_hundred", "total_boosters_per_hundred",
    "new_vaccinations_smoothed_per_million", "new_people_vaccinated_smoothed",
    "new_people_vaccinated_smoothed_per_hundred", "population_density",
    "median_age", "aged_65_older", "aged_70_older", "cardiovasc_death_rate",
    "diabetes_prevalence", "female_smokers", "male_smokers",
    "life_expectancy", "population"
]

# Removes all other columns
df_covid = df_covid_removed_columns[columns_we_want_to_keep]

In [ ]:
# Check if the columns were removed
df_covid.info()

In [ ]:
# Check how the dataset look and how we should proceed
df_covid

In [ ]:
# Shape of the dataframe after some cleaning
print(f"COVID dataframe shape after removing both some columns and rows: {df_covid.shape}")

#### 1.3.1 Separating the continent-level data into its own DataFrame 

We are separating the continent-level data into its own DataFrame so that we can clean and process it independently from the country-level data. This allows us to apply different cleaning steps based on the nature of the data, since continent aggregates may have different structures or missing values compared to individual countries.

In [ ]:
# Before removing the iso_code column, we want to secure the OWID fields for the continents since it could be relevant data to analyze.
rows_to_secure = ["OWID_AFR", "OWID_ASI", "OWID_EUR", "OWID_EUN", "OWID_NAM", "OWID_OCE", "OWID_SAM"]
df_continents = df_covid[df_covid["iso_code"].isin(rows_to_secure)]

In [ ]:
# Check if the rows were secured
df_continents

We now have a new seperate dataframe called `df_continent` that contains the continent-level data. This DataFrame will be used for further analysis and modeling, while the original `df_covid` DataFrame will focus on country-level data.

In [ ]:
# Remove the columns that are irrelavnt since they containt no values at all like; population_density, median_age
df_continents_romved_columns = df_continents.dropna(axis=1, how='all')
df_continents_romved_columns

In [ ]:
# Check whether the columns were removed
print(f"df_continent shape after removing columns: {df_continents_romved_columns.shape}")
print(f"Removed {df_continents.shape[1] - df_continents_romved_columns.shape[1]} columns from the dataframe.")

The columns `new_vaccinations_smoothed_per_million`, `new_people_vaccinated_smoothed` and `new_people_vaccinated_smoothed_per_hundred` contain a lot of missing values so they are not necessary for our analysis and we will drop them from the `df_continents_cleaned` DataFrame. By removing them, we can simplify the DataFrame and focus on the most relevant features for our analysis.

In [ ]:
# Removing columns there are irrelevant 
df_continents_cleaned = df_continents_romved_columns.drop(['iso_code', 'new_vaccinations_smoothed_per_million', 'new_people_vaccinated_smoothed', 'new_people_vaccinated_smoothed_per_hundred'], axis=1)
df_continents_cleaned

In [ ]:
# Check whether the columns were removed
print(f"df_continent shape after removing columns: {df_continents_cleaned.shape}")
print(f"Removed {df_continents_romved_columns.shape[1] - df_continents_cleaned.shape[1]} columns from the dataframe.")

In [ ]:
# Check for duplicates in the DataFrame
df_continents_cleaned.duplicated().sum()

We still have some rows with missing values in the df_continents_cleaned DataFrame, so we will impute them to ensure that our analysis is accurate and meaningful. This step is important because missing values can lead to biased results or errors in our models.

Since it is a small dataframe, we can't delete the row with missing values(NaN), since we will lose a lot of data. We choose to replace the missing values, even though it can have a high risk of giving wrong information and have a big impact. 

The first four we replaced using realistic data.
But since it took too much time, we’ll use the median to fill the remaining NaN values.

In [ ]:
# method for replacing cell with a value
def replace_cell(df, row_filter, column, value):
    df.loc[row_filter, column] = value

# replace NaN for total_vaccinations for Africa. 1.084.500.000 is from Africa CDC, which is offical and reliable.
replace_cell(df_continents_cleaned, df_continents_cleaned['location'] == 'Africa', 'total_vaccinations', 1084500000)

# replace NaN for total_vaccinations for South America. 970.800.000 is from WTO-IMF COVID-19 Vaccine Trade Tracker, which is offical and reliable.
replace_cell(df_continents_cleaned, df_continents_cleaned['location'] == 'South America', 'total_vaccinations', 970800000)

# replace NaN for people_vaccinated for Africa. 725.000.000 is from Africa CDC, which is offical and reliable
replace_cell(df_continents_cleaned, df_continents_cleaned['location'] == 'Africa', 'people_vaccinated', 725000000)

# replace NaN for people_vaccinated for South America. 351.310.000 is from Our World in Data which is offical and reliable used by WHO.
replace_cell(df_continents_cleaned, df_continents_cleaned['location'] == 'South America', 'people_vaccinated', 351310000)

# method for replacing cell with median 
def fill_na_with_median(df, column_name):
    median_value = df[column_name].median()
    print(f"Median of '{column_name}': {median_value:.2f}")
    df[column_name].fillna(median_value, inplace=True)


fill_na_with_median(df_continents_cleaned, 'people_fully_vaccinated')
fill_na_with_median(df_continents_cleaned, 'total_boosters')
fill_na_with_median(df_continents_cleaned, 'new_vaccinations')
fill_na_with_median(df_continents_cleaned, 'new_vaccinations_smoothed')
fill_na_with_median(df_continents_cleaned, 'total_vaccinations_per_hundred')
fill_na_with_median(df_continents_cleaned, 'people_vaccinated_per_hundred')
fill_na_with_median(df_continents_cleaned, 'people_fully_vaccinated_per_hundred')
fill_na_with_median(df_continents_cleaned, 'total_boosters_per_hundred')
df_continents_cleaned

#### 1.3.2 Isolating the remaining rows in the `df_covid` dataframe

We are isolating the remaining rows in the df_covid DataFrame to ensure it contains only country-level data. This allows us to clean the dataset and retain only the features that are most relevant for our analysis.

In [ ]:
# Since we seperated the OWID continent fields into it's own dataframe earlier, we now have to remove them again for the df_covid dataframe.
rows_to_remove = ["OWID_AFR", "OWID_ASI", "OWID_EUR", "OWID_EUN", "OWID_NAM", "OWID_OCE", "OWID_SAM"]
df_covid_removed_rows = df_covid[~df_covid['iso_code'].isin(rows_to_remove)]
df_covid_removed_rows             

In [ ]:
# Check whether the rows were removed
print(f"COVID dataframe shape after removing columns: {df_covid_removed_rows.shape}")
print(f"Removed {df_covid.shape[0] - df_covid_removed_rows.shape[0]} columns from the dataframe.")

The columns such as `iso_code`, `total_vaccinations`, `population_density`, `aged_70_older`, `female_smokers` and others either contain no values or have a high number of missing entries. Since our analysis focuses on deaths and infections in relation to population, these columns are not essential. Therefore, we will remove them from the `df_covid_removed_rows` DataFrame to simplify the dataset and concentrate on the most relevant features.

In [ ]:
# Remove columns there are irrelavnt from df_covid_cleaned
df_covid_cleaned = df_covid_removed_rows.drop([
    "iso_code","total_vaccinations", "people_vaccinated", "people_fully_vaccinated",
    "total_boosters", "new_vaccinations", "new_vaccinations_smoothed",
    "total_vaccinations_per_hundred", "people_vaccinated_per_hundred",
    "people_fully_vaccinated_per_hundred", "total_boosters_per_hundred",
    "new_vaccinations_smoothed_per_million", "new_people_vaccinated_smoothed",
    "new_people_vaccinated_smoothed_per_hundred", "population_density", "median_age", 
    "aged_65_older", "aged_70_older", "cardiovasc_death_rate", "diabetes_prevalence", 
    "female_smokers", "male_smokers"], axis=1)
df_covid_cleaned

In [ ]:
# Check whether the columns were removed
print(f"COVID dataframe shape after removing columns: {df_covid_cleaned.shape}")
print(f"Removed {df_covid_removed_rows.shape[1] - df_covid_cleaned.shape[1]} columns from the dataframe.")

In [ ]:
# Check for duplicates in the DataFrame
df_covid_cleaned.duplicated().sum()

We still have some rows with missing values in the `df_covid_cleaned` DataFrame, so we will impute them with median to ensure that our analysis is accurate and meaningful. This step is important because missing values can lead to biased results or errors in our models.

In [ ]:
# Function to fill NaN values with the median of the specified column
def fill_na_with_median(df, column_name):
    median_value = df[column_name].median()
    print(f"Median of {column_name}: {median_value:.2f}")
    df[column_name].fillna(median_value, inplace=True)

# Fill NaN values with the median for the columns; total_cases, total_deaths, total_cases_per_million, total_deaths_per_million and life_expectancy
fill_na_with_median(df_covid_cleaned, "total_cases")
fill_na_with_median(df_covid_cleaned, "total_deaths")
fill_na_with_median(df_covid_cleaned, "total_cases_per_million")
fill_na_with_median(df_covid_cleaned, "total_deaths_per_million")
fill_na_with_median(df_covid_cleaned, "life_expectancy")


### 1.4 Explore the new cleaned dataframes

#### 1.4.1 Explore df_continents_cleaned dataset

In [ ]:
# Check the shape of the DataFrame (rows, columns)
df_continents_cleaned.shape

In [ ]:
# Gives an overview of the DataFrame
df_continents_cleaned.info()

In [ ]:
# List the columns in the DataFrame
list(df_continents_cleaned)

In [ ]:
# Gives summary statistics for all numerical columns in the dataset
df_continents_cleaned.describe()

##### **1.4.1.1 Check for outliers in the df_continents_cleaned**

The next step in exploring the data is checking for outlier values that are unusually high or low compared to the rest of the data.

We use the IQR (Interquartile Range) method, which is a common way to detect outliers:

-  First, we calculate the first quartile (Q1) and third quartile (Q3) for each selected column.
- The IQR is the difference between Q3 and Q1.
- Any value that falls below Q1 - 1.5 * IQR or above Q3 + 1.5 * IQR is considered an outlier.

In [ ]:
df_continents_cleaned

In [ ]:
# Check for outliers in df_continents_cleaned using IQR method
print("\n..Checking for outliers in df_continents_cleaned:")

# Loop through selected columns
for column in ['total_cases_per_million', 'total_deaths_per_million', 'people_vaccinated_per_hundred']:
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df_continents_cleaned[column].quantile(0.25)
    Q3 = df_continents_cleaned[column].quantile(0.75)
    IQR = Q3 - Q1  # Interquartile Range

    # Define the lower and upper bounds for detecting outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Find rows where the value is outside the normal range
    outliers = df_continents_cleaned[
        (df_continents_cleaned[column] < lower_bound) | 
        (df_continents_cleaned[column] > upper_bound)
    ]

    # Print the number of outliers found for the column
    print(f"  {column}: {len(outliers)} outliers detected")

In [ ]:
# the outlier there was detected
print(outliers[['location', column]])


**1.4.1.2 Conclusion of outliers**:
Only one outlier was detected in the people_vaccinated_per_hundred column.
This indicates that one continent(Oceania) slightly deviates in terms of vaccination coverage compared to the others.
However the value is not far from the overall range, so it will be kept in the dataset.

##### **1.4.1.3 Visualize the Dataset Statistics - df_continents_cleaned**

In [ ]:
# histograms for df_continents_cleaned
columns = ['total_cases_per_million', 'total_deaths_per_million', 'people_vaccinated_per_hundred']
axes = df_continents_cleaned[columns].hist(figsize=(10, 5))
for ax, col in zip(axes.flatten(), columns):
    ax.set_xlabel(col + " (value)")         
    ax.set_ylabel("number of continents") 

plt.tight_layout()
plt.show()

Above histogram shows following:

Total cases per million:
Most continents have between 300,000 and 400,000 cases per million.

Total deaths per million:
The death rate is higher in a few continents, suggesting potential differences in healthcare or reporting.

People vaccinated per hundred:
Most continents have similar vaccination coverage, except Oceania, which is lower and may indicate slower rollout.

In [ ]:
# bar chart for df_continents_cleaned
sns.barplot(x='location', y='total_deaths_per_million', data=df_continents_cleaned)
plt.title('Total COVID-19 deaths per million by continent')
plt.xlabel('Continent')
plt.ylabel('Deaths per million')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Above bar chart shows there are clear differences in COVID-19 deaths per million across continents. 
South America has the highest death rate, followed closely by Europe and North America. 
In contrast, Africa and Asia show much lower death rates, which may reflect differences in healthcare systems, 
demographics, or data reporting.

#### 1.4.2 Explore df_covid_cleaned dataset

In [ ]:
# Check the shape of the DataFrame (rows, columns)
df_covid_cleaned.shape

In [ ]:
# Gives an overview of the DataFrame
df_covid_cleaned.info()

In [ ]:
# Display a random sample of 10 rows from the DataFrame
df_covid_cleaned.sample(10)

In [ ]:
# List the columns in the DataFrame
list(df_covid_cleaned)

In [ ]:
# Gives summary statistics for all numerical columns in the dataset
df_covid_cleaned.describe()

Now that we explored the new cleaned dataframe a bit, we can see that the `df_covid_cleaned` DataFrame contains a more manageable number of columns and rows vs the original dataframe. The columns we have retained are relevant for our analysis, and we have removed unnecessary or redundant features.

##### **1.4.2.1 Check for outliers in the df_covid_cleaned**

We apply this method to four important features in both datasets: total_cases, total_deaths, life_expectancy and population. This helps us find any unusual data points that could affect the results of our analysis.

In [ ]:
# Check for outliers in covid dataset using IQR method
print("\n..Checking for outliers in the covid dataframe:")

# Loop through selected columns
for column in ['total_cases', 'life_expectancy', 'population']:
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df_covid_cleaned[column].quantile(0.25)
    Q3 = df_covid_cleaned[column].quantile(0.75)
    IQR = Q3 - Q1  # Interquartile Range

    # Define the lower and upper bounds for detecting outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Find rows where the value is outside the normal range
    outliers = df_covid_cleaned[
        (df_covid_cleaned[column] < lower_bound) | 
        (df_covid_cleaned[column] > upper_bound)
    ]

    # Print the number of outliers found for the column
    print(f"  {column}: {len(outliers)} outliers detected")


**1.4.2.2 Conclusion of outliers**: The dataset contains outliers across several features — particularly in total_cases (39 outliers) and population (25 outliers).

These outliers are likely not errors but reflect extreme yet valid data points related to the real impact of COVID-19 in certain countries. For this reason, we’ve chosen to keep them. These values could provide valuable insights into the factors that influenced high case and death counts. Removing them might hide important patterns in the data.


##### **1.4.2.3 Scaling**

In [ ]:
# get statistics
scaled_data = df_covid_cleaned[['total_deaths_per_million']]

print('Mean:', scaled_data['total_deaths_per_million'].mean())
print('Standard Deviation:', scaled_data['total_deaths_per_million'].std())

In [ ]:
# draw histogram to visualize them
sns.histplot(scaled_data['total_deaths_per_million'], color='#ee4c2c', bins=50);
plt.ylabel("Number of countries")
plt.show()

As the graph above shows, most countries have fewer than 1000 deaths per million, but a few have significantly higher numbers. This creates a right-skewed distribution, possibly reflecting differences in healthcare systems or pandemic responses.

##### **1.4.2.4 Standard Scalling**

In [ ]:
# reduce all with the mean and scale the data to unit variance
# x = (x-xmean)/std
standard_scaler = StandardScaler()
scaled_data['total_deaths_per_million'] = standard_scaler.fit_transform(scaled_data[['total_deaths_per_million']])

print('Mean:', scaled_data['total_deaths_per_million'].mean()) # almost 0
print('Standard Deviation:', scaled_data['total_deaths_per_million'].std()) # almost 1

In [ ]:
# histogram has same shape, but 0,0 is in the middle
plt.figure(figsize=(12, 4))
sns.histplot(scaled_data['total_deaths_per_million'], color='#ee4c2c', bins=50);
plt.ylabel("Number of countries")
plt.tight_layout()
plt.show()

After standard scaling, the distribution shape remains the same, but values are now centered around 0 with a standard deviation of 1. This makes the data easier to compare with other features and is especially useful for machine learning models that are sensitive to different value ranges. 

##### **1.4.2.5 Min-Max Scalling - Normalization**

In [ ]:
minmax_scaler = MinMaxScaler()
scaled_data['death_min_max_scaled'] = minmax_scaler.fit_transform(scaled_data[['total_deaths_per_million']])

print('Mean:', scaled_data['death_min_max_scaled'].mean())
print('Standard Deviation:', scaled_data['death_min_max_scaled'].std())

In [ ]:
# values are in [0, 1]
sns.histplot(scaled_data['death_min_max_scaled'], color='#ee4c2c', bins=50);
plt.ylabel("Number of countries")
plt.show()

In [ ]:
qtrans = QuantileTransformer()
scaled_data['death_trans_uniform'] = qtrans.fit_transform(scaled_data[['total_deaths_per_million']])

print('Mean:', scaled_data['death_trans_uniform'].mean())
print('Standard Deviation:', scaled_data['death_trans_uniform'].std())

In [ ]:
qtrans = QuantileTransformer()
scaled_data['death_trans_uniform'] = qtrans.fit_transform(scaled_data[['total_deaths_per_million']])

print('Mean:', scaled_data['death_trans_uniform'].mean())
print('Standard Deviation:', scaled_data['death_trans_uniform'].std())

In [ ]:
sns.boxplot(x='continent', y='total_deaths_per_million', data=df_covid_cleaned)
plt.title("Total deaths per million by continent")
plt.xticks(rotation=45)
plt.show()

Above boxplot shows that South America and Europe have the highest median deaths per million, while Africa and Asia have the lowest. There are many outliers, especially in Africa, indicating large variation between countries within continents.

##### **1.4.2.6 Correlation Matrix**

In [ ]:
correlation_matrix = df_covid_cleaned.select_dtypes(include='number').corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation matrix over COVID-19 and related factors")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

There is a weak correlation (−0.07) between population size and total deaths per million, suggesting that population alone does not explain differences in COVID-19 death rates. Other factors like healthcare quality or pandemic response may play a larger role.

##### **1.4.2.7 Checking for Normal Distribution for the df_covid_cleaned**

##### Checking for Normal Distribution
Before applying statistical methods, it is important to understand how the data is distributed. Many techniques — such as t-tests, ANOVA, and linear regression — assume that the data follows a normal distribution. If this assumption is not met, the results may not be reliable.

To check for normality, we define a custom function called check_normality. This function runs through each numeric column (excluding total_deaths) and performs several checks:

**Skewness** measures how symmetric the data is:

- A positive skew means the distribution has a long tail on the right.
- A negative skew means the tail is on the left.

General interpretation:

- |skew| < 0.5 → roughly symmetric
- 0.5 < |skew| < 1 → moderately skewed
- |skew| > 1 → highly skewed

**Kurtosis** measures how sharp or flat the peak is, and how heavy the tails are compared to a normal curve:

- Kurtosis > 3 → sharper peak, more outliers
- Kurtosis < 3 → flatter peak, fewer outliers
- In pandas, the `kurt()` function returns **excess kurtosis**, so a value of 0 indicates a normal distribution.

The function also applies two statistical tests for normality:

- D’Agostino’s K-squared test
- Jarque-Bera test

Both tests return a p-value. If the p-value is greater than 0.05, the data is considered likely to be normally distributed. If the p-value is below 0.05, the data likely deviates from a normal distribution.

Based on these results, each feature is classified as:

- Yes – likely normal (both p-values > 0.05 and skewness < 1)
- Partial – somewhat normal (D’Agostino p > 0.01 and skewness < 2)
- No – not normal (does not meet the above criteria)

The function returns a summary table showing each feature’s skewness, kurtosis, p-values, and classification. This step is an important part of exploratory data analysis and helps us decide whether we need to transform any variables or choose alternative statistical methods.

In [ ]:
print("\n..Test normality in the covid dataframe:")

# Function to test normality of numeric columns
def check_normality(df):
    """
    Tests whether numeric columns follow a normal distribution.
    Uses D’Agostino and Jarque-Bera tests. 
    """
    num_cols = [col for col in df.select_dtypes(include=['float64', 'int64']).columns 
                if col != 'total_deaths']
    
    rows = []

    for col in num_cols:
        data = df[col]
        skewness = data.skew()
        kurtosis = data.kurt()
        dagostino = stats.normaltest(data)
        jb = stats.jarque_bera(data)

        normal = "No"
        if dagostino.pvalue > 0.05 and jb.pvalue > 0.05 and abs(skewness) < 1:
            normal = "Yes"
        elif dagostino.pvalue > 0.01 and abs(skewness) < 2:
            normal = "Partial"

        rows.append({
            'Column': col,
            'Skewness': round(skewness, 3),
            'Kurtosis': round(kurtosis, 3),
            "D'Agostino p-value": f"{dagostino.pvalue:.2e}",
            "Jarque-Bera p-value": f"{jb.pvalue:.2e}",
            'Normally Distributed?': normal
        })

    return pd.DataFrame(rows)

# Run normality checks on all numeric columns
check_normality(df_covid_cleaned)

In [ ]:
def visualize_distributions(df):
    """
    Visualizes the distribution of all numeric columns with histogram and QQ-plot.
    """
    num_cols = [col for col in df.select_dtypes(include=['float64', 'int64']).columns 
                if col != 'total_deaths']
    
    fig, axes = plt.subplots(len(num_cols), 2, figsize=(12, 4 * len(num_cols)))

    for i, col in enumerate(num_cols):
        ax1 = axes[i, 0] if len(num_cols) > 1 else axes[0]
        ax2 = axes[i, 1] if len(num_cols) > 1 else axes[1]

        sns.histplot(df[col], kde=True, ax=ax1)
        ax1.set_title(f'Distribution of {col}')

        stats.probplot(df[col], dist="norm", plot=ax2)
        ax2.set_title(f'QQ-Plot of {col}')

    plt.tight_layout()
    plt.show()

In [ ]:
# Call the function
visualize_distributions(df_covid_cleaned)

##### **1.4.2.8 Summary: Normality of Numeric Variables**

Statistical tests and visualizations show that none of the numeric variables in the dataset are normally distributed. Variables like `total_cases` and `population` are highly right-skewed, while others such as `total_cases_per_million`, `total_deaths_per_million`, and `life_expectancy` show moderate skewness but still fail normality tests.

With a moderate sample size of 235 observations, these results likely reflect genuine distribution patterns rather than test sensitivity. This indicates that analytical methods assuming normality (e.g., parametric tests or linear regression without transformation) may not be suitable without preprocessing steps like normalization or transformation.

##### **1.4.2.9 Feature Selection**
Feature selection and comparison were carried out during the data cleaning and exploratory process. In our analysis, we consider deaths as the dependent variable, while the remaining features are treated as independent variables that may help explain or influence the death toll.

# 2. Supervised machine learning: linear regression

In [ ]:
# visualise the features and the response using scatterplots
sns.pairplot(df_covid_cleaned, x_vars=['total_cases_per_million'], y_vars='total_deaths_per_million', height=5, aspect=1.6)
plt.tight_layout()
plt.show()

In [ ]:
# independent
X = df_covid_cleaned['total_cases_per_million'].values.reshape(-1, 1)
# dependent
y = df_covid_cleaned['total_deaths_per_million'].values.reshape(-1, 1)

In [ ]:
# plot all
plt.ylabel('total_deaths_per_million')
plt.xlabel('total_cases_per_million')
plt.scatter(X, y, color='blue')
plt.show()

The scatter plot shows a weak positive correlation between total COVID-19 cases per million and deaths per million. However, the data is widely spread with several outliers, which reduces the accuracy of a linear regression model. This suggests that the relationship is not strongly linear and may require further data processing or additional features for better prediction.

In [ ]:
df_covid_cleaned.plot.line(subplots=True)

In [ ]:
sns.lmplot(x='total_cases_per_million',y='total_deaths_per_million',data=df_covid_cleaned,fit_reg=True) 

The scatter plot with regression line shows a weak positive relationship between total COVID-19 cases and deaths per million. Although the trend is upward, the wide confidence interval and spread of points indicate high variability, suggesting the linear model has limited predictive power.

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123, test_size=0.15) 

In [ ]:
# the shape of the subsets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

### Train a ML Model

In [ ]:
# creating an instance of Linear Regression model
myreg = LinearRegression()

In [ ]:
# fit it to our data
myreg.fit(X_train, y_train)
myreg

In [ ]:
# get the calculated coefficients
a = myreg.coef_
b = myreg.intercept_

In [ ]:
a

In [ ]:
b

In [ ]:
print(f"The model is a line, y = a * x + b, or y = {a} * x + {b}")


### Test the data

In [ ]:
y_predicted = myreg.predict(X_test)

In [ ]:
# Visualise the Linear Regression 
plt.title('Linear Regression')
plt.scatter(X, y, color='green')
plt.plot(X_train, a*X_train + b, color='blue')
plt.plot(X_test, y_predicted, color='orange')
plt.xlabel('total_cases_per_million')
plt.ylabel('total_deaths_per_million')
plt.show()

The regression plot shows a weak positive correlation between COVID-19 cases and deaths per million. The blue line represents the model's fitted trend, while orange dots indicate predictions. The high spread of actual values around the line suggests limited predictive accuracy of the linear regression model.

In [ ]:
#predict age from length
death_predicted = myreg.predict([[170]])
death_predicted

In [ ]:
death_predict = a * 170 + b
death_predict

### unknown data 

In [ ]:
length = 145
death_predicted = myreg.predict([[length]])
death_predicted

The model's predictive power is limited, it can still be used to estimate the expected number of deaths per million based on new case data. For example, given 145,000 cases per million, the model predicts approximately 724 deaths per million. However, due to high variance in the data, this should only be interpreted as a rough estimate.

### Model Evaluation

In [ ]:
# MAE
mae = metrics.mean_absolute_error(y_test, y_predicted)
print(mae)

In [ ]:
# MSE
mse = metrics.mean_squared_error(y_test, y_predicted)
print(mse)

In [ ]:
# RMSE
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_predicted))
print(rmse)

MAE is the easiest to understand, because it's the average error, measured in the same units like the data
MSE is more popular than MAE, because MSE amplifies larger errors, making it useful when larger errors are particularly costly
RMSE is even more popular than MSE, because RMSE combines the benefits of both MSE and MAE

In [ ]:
# Explained variance score: the proportion of the variance in a dependent variable that can be explained by the model
# 1 for perfect prediction
eV = round(sm.explained_variance_score(y_test, y_predicted), 2)
print('Explained variance score ',eV )

In [ ]:
# R-squared: the proportion of the variation in the dependent variable that is predictable from the independent variable(s)
#r2_score(y, predict(X))
#r2_score(y_test, y_predicted)

# 3. Supervised machine learning: classification

To investigate what factors may indicate a country’s risk of experiencing a high number of COVID-19 deaths, we trained a Random Forest Classifier using total cases, life expectancy, and population as input features. A binary target variable, high_mortality, was created by labeling countries with total deaths above the median as "high mortality" and those below as "low mortality".

In [ ]:
# Binary target based on the median
median_deaths = df_covid_cleaned['total_deaths'].median()
df_covid_cleaned['high_mortality'] = df_covid_cleaned['total_deaths'] > median_deaths

In [ ]:
# choose feature and target 
features = ['total_cases', 'life_expectancy', 'population']
target = 'high_mortality'

X = df_covid_cleaned[features]
y = df_covid_cleaned[target]

# Split i træning og test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

We tested the model using a 75/25 split between training and test data. The model achieved an accuracy of 88.1%, indicating high precision and strong generalization ability. A previous test using an 80/20 split gave 87.2%, confirming the model’s stability across different data splits. Precision will be 0.85 (low mortality), 0.91 (high mortality)

In [ ]:
# Get and sort the feature importance 
importances = model.feature_importances_
features_df = pd.DataFrame({
    'Feature': features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(8, 5))
sns.barplot(data=features_df, x='Importance', y='Feature')
plt.title('Feature Importance (Random Forest Classifier)')
plt.tight_layout()
plt.show()

# 4. Unsupervised machine learning: clustering

To uncover patterns in how countries were affected by COVID-19, we applied KMeans clustering using the features:
total_cases, life_expectancy, and population.

In [ ]:
# choose numeric features for clustering
features_cluster = ['total_cases', 'life_expectancy', 'population']
X_cluster = df_covid_cleaned[features_cluster]

# Standardise data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

In [ ]:
# KMeans cluster
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

# add cluster labels for dataframe
df_covid_cleaned['cluster'] = clusters


In [ ]:
# Silhouette score
score = silhouette_score(X_scaled, clusters)
print(f"Silhouette Score: {score:.2f}")


Before clustering, we standardized the data using StandardScaler to ensure equal weighting of features.

We chose to use KMeans with 3 clusters, which grouped countries based on similarities in the selected variables. The silhouette score was 0.53, indicating a reasonably good separation between clusters

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(
    x='total_cases_per_million',
    y='life_expectancy',
    hue='cluster',
    data=df_covid_cleaned,
    palette='Set2'
)
plt.title('KMeans Clustering of Countries')
plt.xlabel('Cases per Million')
plt.ylabel('Life Expectancy')
plt.legend(title='Cluster')
plt.tight_layout()
plt.show()

The clusters were visualized in a 2D scatterplot showing how countries separate based on life expectancy and COVID-19 cases. The results showed:
Cluster 1 (168 countries): moderate life expectancy and high case numbers
Cluster 2 (64 countries): generally lower life expectancy and lower case numbers
Cluster 0 (3 countries): outliers with extreme combinations

In [ ]:
inertias = []
k_values = range(1, 10)

for k in k_values:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X_scaled)
    inertias.append(km.inertia_)

plt.figure(figsize=(6, 4))
plt.plot(k_values, inertias, marker='o')
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia")
plt.title("Elbow Method for Optimal K")
plt.tight_layout()
plt.show()


## Summarize

As part of our MP3 project, we realized that we need to gather more data for our exam project to better address our hypotheses about factors influencing COVID-19 death rates. However, based on our current analysis — including the correlation matrix — we can conclude that there is a weak correlation (−0.07) between population size and total deaths per million. This suggests that population alone does not explain the differences in death rates; other factors such as healthcare quality or pandemic response may play a more important role, but further data is needed to confirm this.